## Машинное обучение. Random Forest.

### Практика 6.

### Задачи: 
Используя данные из файла https://github.com/mcsml/ml_basics/blob/master/bioresponse.csv, обучить случайный лес (random forest).
**Target**-колонка называется *Activity* (эти данные надо предсказывать)

**D1 - D1770** - features.

Дополнительные задания:

1. Обучить *XGBClassifier* на этих данных.
2. Поэкспериментировать с количеством моделей в композиции (n_estimators) и глубиной деревьев (max_depth).
3. Построить графики зависимости ошибки от этих гиперпараметров для случайного леса и XGBClassifier.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df_bio = pd.read_csv(r'https://raw.githubusercontent.com/mcsml/ml_basics/master/bioresponse.csv')

In [3]:
df_bio

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3746,1,0.033300,0.506409,0.10,0.0,0.209887,0.633426,0.297659,0.376124,0.727093,...,0,0,0,0,0,0,0,0,0,0
3747,1,0.133333,0.651023,0.15,0.0,0.151154,0.766505,0.170876,0.404546,0.787935,...,0,0,1,0,1,0,1,0,0,0
3748,0,0.200000,0.520564,0.00,0.0,0.179949,0.768785,0.177341,0.471179,0.872241,...,0,0,0,0,0,0,0,0,0,0
3749,1,0.100000,0.765646,0.00,0.0,0.536954,0.634936,0.342713,0.447162,0.672689,...,0,0,0,0,0,0,0,0,0,0


1. Делим на тестовые и тренировочные данные

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
x = df_bio.drop(['Activity'], axis = 1) #делим данные на features X и lables Y
y = df_bio['Activity']

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [7]:
x_test #проверим, как разделилось

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
2148,0.100000,0.734427,0.20,0.25,0.136259,0.674081,0.212153,0.512164,0.707647,0.244693,...,0,0,0,0,0,0,0,0,0,0
1044,0.000000,0.372990,0.00,0.00,0.124049,0.608342,0.405229,0.593604,0.737318,0.368695,...,0,0,0,0,0,0,0,0,0,0
3319,0.000000,0.337305,0.00,0.00,0.192214,0.702996,0.281719,0.067800,0.805263,0.191363,...,0,0,0,0,0,0,0,0,0,0
439,0.033300,0.560480,0.05,0.00,0.154026,0.616952,0.228655,0.528427,0.722801,0.317141,...,0,0,0,0,0,0,0,0,0,0
3516,0.033300,0.694001,0.00,0.00,0.280807,0.574624,0.451219,0.705057,0.707184,0.410486,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2403,0.033300,0.428737,0.00,0.00,0.140261,0.815605,0.214042,0.308958,0.880726,0.124487,...,0,0,0,0,0,0,0,0,0,0
3127,0.133333,0.576083,0.00,0.00,0.159228,0.646690,0.329312,0.569801,0.743771,0.324600,...,0,0,0,0,0,0,0,0,0,0
612,0.000000,0.497009,0.10,0.00,0.180349,0.629059,0.295410,0.408834,0.733564,0.312651,...,0,0,0,0,0,0,0,0,0,0
2677,0.000000,0.751084,0.00,0.00,0.363752,0.578991,0.469010,0.514104,0.642857,0.459723,...,0,0,0,0,0,0,0,0,0,0


2. Подгружаем инструменты scikit-learn и модель random forest

In [8]:
from sklearn.ensemble import RandomForestClassifier

In [9]:
clf = RandomForestClassifier(n_estimators = 40, max_depth = 10, random_state = 1)
clf.fit(x_train, y_train)

RandomForestClassifier(max_depth=10, n_estimators=40, random_state=1)

3. Проверяем модель на тестовых данных

In [10]:
from sklearn.metrics import accuracy_score
y_pred = clf.predict(x_test)
accuracy_score(y_pred, y_test)

0.8029294274300932

4. Проверяем модель на тренировочных данных

In [11]:
y_pred = clf.predict(x_train)
accuracy_score(y_train, y_pred)

0.9273333333333333

5. Теперь пробуем проделать тоже самое, но при помощи XGBClassifier

In [12]:
conda install -c conda-forge py-xgboost

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda3

  added / updated specs:
    - py-xgboost


The following NEW packages will be INSTALLED:

  _py-xgboost-mutex  conda-forge/win-64::_py-xgboost-mutex-2.0-cpu_0
  libxgboost         pkgs/main/win-64::libxgboost-1.5.0-hd77b12b_2
  py-xgboost         pkgs/main/win-64::py-xgboost-1.5.0-py39haa95532_2
  python_abi         conda-forge/win-64::python_abi-3.9-2_cp39

The following packages will be UPDATED:

  conda              pkgs/main::conda-4.12.0-py39haa95532_0 --> conda-forge::conda-22.9.0-py39hcbf5309_1



Note: you may need to restart the kernel to use updated packages.Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done





==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 22.9.0

Please update conda by running

    $ conda update -n base -c defaults conda




In [13]:
conda update -n base -c defaults conda

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda3

  added / updated specs:
    - conda


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _py-xgboost-mutex-2.0      |            cpu_0          12 KB
    ------------------------------------------------------------
                                           Total:          12 KB

The following packages will be REMOVED:

  python_abi-3.9-2_cp39
  xmltodict-0.12.0-pyhd3eb1b0_0

The following packages will be UPDATED:

  conda-package-han~                   1.7.3-py39h8cc25b3_1 --> 1.9.0-py39h8cc25b3_0 None
  pyjwt                                2.1.0-py39haa95532_0 --> 2.4.0-py39haa95532_0 None

The following packages will be SUPERSEDED by a higher-priority channel:

  _py-xgboost-mutex                             conda-forge --> pkgs/main None
  conda              conda-forge::cond

In [14]:
import xgboost as xgb

In [15]:
gr_boost = xgb.XGBClassifier(n_estimators = 150, max_depth = 2, random_state = 1)

In [17]:
gr_boost.fit(x_train, y_train)

Z = gr_boost.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.plot(x_train[y_train == 0][:, 0], x_train[y_train == 0][:, 1], 'ro')
plt.plot(x_train[y_train == 1][:, 0], x_train[y_train == 1][:, 1], 'bo')
plt.plot(x_train[y_train == 2][:, 0], x_train[y_train == 2][:, 1], 'yo')
plt.contourf(xx, yy, Z, cmap=light_colors, alpha=0.8)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:53:00] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


NameError: name 'xx' is not defined

5. Визуализация данных

In [12]:
x_train = np.array(x_train) #переводим датафреймы в массивы со списками данных для визуализации
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [13]:
h = 0.01
x_min, x_max = x_train[:, 0].min()-1, x_train[:, 0].max()+1
y_min, y_max = x_train[:, 1].min()-1, x_train[:, 1].max()+1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

In [15]:
Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.plot(x_train[y_train == 0][:, 0], x_train[y_train == 0][:, 1], 'ro')
plt.plot(x_train[y_train == 1][:, 0], x_train[y_train == 1][:, 1], 'bo')
plt.plot(x_train[y_train == 2][:, 0], x_train[y_train == 2][:, 1], 'yo')
plt.contourf(xx, yy, Z, cmap=light_colors, alpha=0.8)

ValueError: X has 2 features, but DecisionTreeClassifier is expecting 1776 features as input.